In [1]:
# packages
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import glob
from matplotlib.patches import Patch
from matplotlib.lines import Line2D

In [36]:
historical = pd.read_excel(r"../Nameplate_Capacity/EIA860_2020_nameplate_statistics_natl_plant_post1990.xlsx")
tech_spec = pd.read_excel("tech_specs.xlsx",sheet_name="originals")

In [47]:
def calc_nameplate(state_name,tech_name,cap_scen):
    # technologies modified by user are not eligible
    this_row = historical.query("ReEDS_technology==@tech_name").query("State==@state_name")
    if "pv" in tech_name:
        tech_name = "pv"
    # reformat
    translate_table = str.maketrans({' ': '-'})
    tech_name = tech_name.translate(translate_table)
    
    if len(this_row) == 1:
        # the technology exists in the list for the state
        mid_val = this_row["Median"].iloc[0]
        low_val = this_row["LowerQuartile"].iloc[0]
        high_val = this_row["HigherQuartile"].iloc[0]
    elif len(historical.query("ReEDS_technology==@tech_name"))> 0 and len(this_row) == 0:
        # the technology exists but not for the state, assume natl avg
        natl_row = historical.query("ReEDS_technology==@tech_name").query("State=='Natl'")
        mid_val = natl_row["Median"].iloc[0]
        low_val = natl_row["LowerQuartile"].iloc[0]
        high_val = natl_row["HigherQuartile"].iloc[0]
    elif len(historical.query("ReEDS_technology==@tech_name")) == 0:
        # the technology doesn't exist, use default JEDI value
        tech_orig = tech_spec.query("tech==@tech_name")
        mid_val = tech_orig["orginal MW"].iloc[0]
        low_val = tech_orig["orginal MW"].iloc[0]
        high_val = tech_orig["orginal MW"].iloc[0]

    if cap_scen == 'mid':
        output_val = mid_val
    elif cap_scen == 'low':
        output_val = low_val
    elif cap_scen == 'high':
        output_val = high_val

    if "pv" in tech_name or "battery" in tech_name:
        output_val = output_val*1000

    return output_val